<div> <div style="width:75%;float: left; " > <H1 style = "font-size:62px;color:#8B0000;">Stellar Oscillations </H1> </div> <div style="width:25%;float: right;" > <img style="float: right;" src="https://www.asterochronometry.eu/images/Asterochronometry_full.jpg" width="90%" /> </div> </div>

The surface of stars like the Sun oscillate as a result of sound waves that propagate through the stellar interior. We explore some of the basic properties of these sound waves here.

Towards the centre of the star, the gas gets both warmer and denser. As a result the sound speed changes with depth and the soundwaves are refracted. We show this with the arrows in the plot below. The innermost radius that the sound waves reach before they return to the surface is indicated with a dotted line in the corresponding colour. You can vary the degree and the frequency of the oscillations to see how this affects the inner turning point (red circle).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
#rcParams['text.usetex'] = True
import ipywidgets as widgets
from matplotlib import rc
from ipywidgets import interactive
from ipywidgets import interact, interact_manual
import matplotlib.patches as patches
import matplotlib
from matplotlib.colors import LightSource
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import numpy as np
import scipy.special as sp
#from mpl_toolkits.basemap import Basemap
import pandas as pd
import os, imageio
from ipywidgets import interactive
from ipywidgets import interact, interact_manual
#import warnings; warnings.simplefilter('ignore')
#import textwrap as tw
rcParams['animation.html'] = 'html5'

In [2]:
def drawsphere(l,m,rotation=0,cent_lat=45,cent_lon=-100):

    # This function is originally found on https://stackoverflow.com/questions/34397418/python-combine-plots-in-grid
    # Only slightly adapted
    
    res = np.pi/250 # resolution

    theta = np.r_[0:2*np.pi:res]+rotation
    phi = np.r_[0:np.pi:res] # theta: lon, phi: coalt

    coef = []
    for i in theta:
        for j in phi:
            coef.append(sp.sph_harm(m,l,i,j))
    coef = np.asarray(coef) # convert list to array
    coef = np.reshape(coef, (len(theta),-1)) # reshapte array as per number of angles

    ## Plotting ##

    # create latitude/longitude arrays
    lon = np.linspace(0,2*np.pi,len(theta))
    lat = np.linspace(-np.pi/2,np.pi/2,len(phi))
    colat = lat+np.pi/2 # colatitude array
    # create 2D meshgrid
    mesh_grid = np.meshgrid(lon, lat) # create a meshgrid out of lat/lon
    lon_grid = mesh_grid[0] # grab the meshgrid part for lon
    lat_grid = mesh_grid[1] # grab the meshgrid part for lat

    real_coef = np.real(coef) # read parts of the coefficients
    norm_coef = np.round(real_coef / np.max(real_coef),2) # normalize
    # set up orthographic map projection
    mp = Basemap(projection='ortho', lat_0 = cent_lat, lon_0 = cent_lon) 
    # setup an orthographic basemap centered at lat_0 & lon_0
    # draw the edge of the map projection region (the projection limb)
    mp.drawmapboundary()

    # convert angles from radians to degrees & pipe them to basemap
    x,y = mp(np.degrees(lon_grid), np.degrees(lat_grid)) 
    # cax = figure.add_axes([0.15,0.03,0.7,0.03])
#         cb = plt.colorbar(orientation = 'horizontal')
    return mp,x, y, norm_coef

def gif_maker(gif_name,png_dir,gif_indx,num_gifs,dpi=90):

    # This function is taken from https://github.com/makerportal/rotating_globe
    # Only very minor changes
    
    if gif_indx==num_gifs-1:
        # sort the .png files based on index used above
        images,image_file_names = [],[]
        for file_name in os.listdir(png_dir):
            if file_name.endswith('.png'):
                image_file_names.append(file_name)       
        sorted_files = sorted(image_file_names, key=lambda y: int(y.split('_')[1]))

        # define some GIF parameters
        
        frame_length = 0.1 # seconds between frames
        end_pause = 0.1 # seconds to stay on last frame
        # loop through files, join them to image array, and write to GIF called 'wind_turbine_dist.gif'
        for ii in range(0,len(sorted_files)):       
            file_path = os.path.join(png_dir, sorted_files[ii])
            if ii==len(sorted_files)-1:
                for jj in range(0,int(end_pause/frame_length)):
                    images.append(imageio.imread(file_path))
            else:
                images.append(imageio.imread(file_path))
        # the duration is the time spent on each image (1/duration is frame rate)
        imageio.mimsave(gif_name, images,'GIF',duration=frame_length)

def drawone(l,m):
    cmap = cm.get_cmap('plasma') # Set color map
    mp, x, y, norm_coef = drawsphere(l,m)
    mp.pcolor(x,y,np.transpose(norm_coef), cmap=cmap)

In [3]:
ModelS = np.genfromtxt("ModelS.dat",skip_header=5)
r = ModelS[:,0]
c = ModelS[:,1]

def computeplt(l,freq):
    
    theta = np.linspace(0, 2*np.pi, 100)

    fig, ax = plt.subplots(1,figsize=(9,9))

    x1 = np.cos(theta)
    x2 = np.sin(theta)

    ax.plot(x1, x2)
    ax.plot(0.5*x1, 0.5*x2,"c--")
    ax.plot(0.85*x1, 0.85*x2,"k--")
    
    omega = freq/1e6*2*np.pi # Angular frequency is 2pi times cyclic frequency
    R = 6.96340e10
    rt = r[abs(R*r-np.sqrt(c**2*l*(l+1)/omega**2)).argmin()]

    x3 = rt*np.cos(theta)
    x4 = rt*np.sin(theta)

    style = "Simple, tail_width=0.5, head_width=4, head_length=8"
     
    kw = dict(arrowstyle=style, color="k")
    
   # set_connectionstyle("arc,angleA=0,armA=30,rad=10")
    
    for i in np.arange(90,-10,-10):
    
        a3 = patches.FancyArrowPatch((x1[i],x2[i]), (x1[i-10],x2[i-10]),
                             connectionstyle="arc3,rad=0.3",**kw)
    
        plt.gca().add_patch(a3)
    
    kw = dict(arrowstyle=style, color="c")
    
    for i in np.arange(95,15,-20):
    
        a3 = patches.FancyArrowPatch((x1[i],x2[i]), (x1[i-20],x2[i-20]),
                             connectionstyle="arc3,rad=0.5",**kw)
    
        plt.gca().add_patch(a3)
    
    a3 = patches.FancyArrowPatch((x1[15],x2[15]), (x1[95],x2[95]),
                             connectionstyle="arc3,rad=0.53",**kw)
    
    plt.gca().add_patch(a3)
    
    plt.xlabel("Radius",fontsize=16)
    plt.ylabel("Radius",fontsize=16)

    ax.plot(x3, x4,"r--")
    
    ax.tick_params(axis='both', which='major', labelsize=16)
    
    ax.set_aspect(1)

setfreq = widgets.FloatSlider(
    value=3090,
    min=1000,
    max=5000,
    step=1,
    description=r'$\mathrm{Freq. \, [\mu Hz]}:$',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f',
)

setl = widgets.FloatSlider(
         value=10,
         min=0,
         max=100,
         step=1,
         description=r'$\mathrm{Degree} \, (\ell):$',
         disabled=False,
         continuous_update=True,
         orientation='horizontal',
         readout=True,
         readout_format='.0f',
         )


_ = interact(computeplt,l=setl,freq=setfreq)


interactive(children=(FloatSlider(value=10.0, description='$\\mathrm{Degree} \\, (\\ell):$', readout_format='.…

For this illustration, we have used a real numeric model of the Sun: [Model S](https://users-phys.au.dk/jcd/solar_models/) by Jørgen Christensen-Dalsgaard.

<h2> Surface Oscillations </h2>

<p> Below we show how different sound waves translate into surface oscillations. Explore this by varying the number 
of nodes found around the equator (<i>m</i>) and the number of nodes found around great circle through the poles (<i>l</i>).
</p>

In [4]:
#setl2 = widgets.IntSlider(
#        value=6,
#        min=0,
#        max=12,
#        description=r'$\ell: $',
#        disabled=False,
#        continuous_update=True,
#        orientation='horizontal',
#        readout=True,
#  )

#setm = widgets.IntSlider(
#        value=setl2.value,
#        min=0,
#        max=max([0,setl2.value-1]),
#        description=r'$m: $',
#        disabled=False,
#        continuous_update=True,
#        orientation='horizontal',
#        readout=True,
#  )

#_ = interact(drawone,l=setl2,m=setm)


#rotation = np.linspace(0,2*np.pi,50)
#png_dir="./png_dir/"
#gif_indx = 0